# Import necassary libraries and files

In [11]:
from pathlib import Path
import pandas as pd
from email import policy
from email.parser import BytesParser
import os
import re
from bs4 import BeautifulSoup
import numpy as np
import nltk.corpus
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import DutchStemmer
import spacy
import dutch_words
lemmaModel = spacy.load('nl_core_news_lg', disable = ['parser','ner'])

# Checked wordlist
#dutchCorpusFile = open(Path(os.getcwd() + '/opentaal-wordlist-master/elements/basiswoorden-gekeurd.txt'))

# Unchecked wordlist
dutchCorpusFile = open(Path(os.getcwd() + '/opentaal-wordlist-master/wordlist.txt'))
dutchCorpusData = dutchCorpusFile.read()
dutchCorpus = dutchCorpusData.replace('\n', '.').split(".")
dutchCorpusFile.close()

# set column width to maximum for better visibility of data
pd.set_option('display.max_colwidth', None)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kerseje\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\kerseje\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Extract content from the emails

In [12]:
# define and print path to .eml files (emails)
pathString = os.getcwd() + '//BrainjarMails'
path = Path(pathString)
pathLength = len(pathString)
print(path)

# grab every file with the extension .eml
email_files = list(path.glob('*.eml'))

# create lists for the names and content of the emails + filecounter
names = []
contents = []
counter = 1
fileCount = len(email_files)
totalCharacterCount = 0

# loop over all found files
for email in email_files:
    
    #open each file in read bytes mode
    with open(email,'rb') as filepointer:
        
        # name is original filename minus the path and extension
        name = filepointer.name[pathLength:-4]
        
        # Parse data from email to message object
        message = BytesParser(policy=policy.default).parse(filepointer)
        
    # pass the plain text from the body of the email to a string variable. If no plain text is availible, 
    # just pass everything in the body
    try:
        content = message.get_body(preferencelist=('plain')).get_content()
    except:
        content = message.get_body().get_content()
    
    # Extract text from any HTML that is present.
    content = BeautifulSoup(content).get_text()
    
    # Remove escape characters (for example newlines)
    escapes = ''.join([chr(char) for char in range(1, 32)])
    #translator = str.maketrans(escapes, ' ')
    #content = content.translate(translator)
    content = re.sub(r'[' + escapes + r']',' ', content)
    
    # Remove any non-ascii characters
    content = content.encode('ascii', errors='ignore').decode()
    
    # Remove websites from mails (maybe not necassary)
    content = re.sub(r'http\S+', '', content)
    
    # Remove extra whitespaces
    content = re.sub(' +', ' ', content)
    
    # Function for removing excess non-alphanumeric characters (and punctuation)
    def RemoveNonAlphanumeric(contentInput, removePunctuation = False):
        # with punctuation
        if removePunctuation == True:
            contentOutput = re.sub(r'[^A-Za-z0-9 ]+', '',contentInput)
        
        # without punctuation
        else:
            contentOutput = re.sub(r'[^A-Za-z0-9 ,?.:;!]+', '',contentInput)
        
        return contentOutput
    
    # Remove non-alphanumeric characters
    content = RemoveNonAlphanumeric(content)
    
    # additional filtering for privacy may be necessary
    content = re.sub(r'(BIC:) [A-Z]*','',content)
    content = re.sub(r'\w*\d\w*', '', content).strip()
    
    # function for text to lower case
    def toLowerCase(contentInput):
        contentOutput = contentInput.lower()
        return contentOutput
    
    # remaining text to lower case
    #content = toLowerCase(content)
    
    # Get amount of characters in all text
    totalCharacterCount += len(content)
    
    # Stopwords removal function
    def StopwordRemoval(contentInput, languageCode):
        stop = stopwords.words(languageCode)
        contentOutput =  " ".join([word for word in contentInput.split() if word not in (stop)])
        return contentOutput
    
    # Stemming function
    def Stemmer(contentInput):
        tokenizedWords = word_tokenize(contentInput, language='dutch')
        stemmedContent = []
        stemmer = DutchStemmer()
        for word in tokenizedWords:
            stemmedContent.append(stemmer.stem(word))
            stemmedContent.append(" ")
        return "".join(stemmedContent)
       
    # Lemmatization function
    def Lemmatizer(contentInput):
        document = lemmaModel(contentInput)
        return " ".join([token.lemma_ for token in document])
    
    # remove stopwords
    #content = StopwordRemoval(content, 'dutch')
    
    # Stemming or lemmatization
    #content = Stemmer(content)
    #content = Lemmatizer(content)
    
    # remove words that are not in a dictionary (dutch in this case)
    content =  " ".join([word for word in content.split() if word in (dutchCorpus)])
    
    # add name and content of current email to their respective lists
    names.append(name)
    contents.append(content)
    
    #close the current file
    filepointer.close()
    
    # filecounter
    print("Counter: " + str(counter) + '/' + str(fileCount), end="\r")
    counter += 1
    
print('Total character count: ' + str(totalCharacterCount))

C:\Users\kerseje\Bachelerproef jupyter notebooks\BrainjarMails


C:\Users\kerseje\AppData\Local\miniconda3\lib\site-packages\bs4\__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


Total character count: 1589750


### Turn lists into dataframe for easy exploration

In [13]:
dfNames = pd.DataFrame([names, contents]).T
dfNames.columns = ['names', 'contents']

### Set class index based on title

In [14]:
dfNames['classIndex'] = 0
dfNames['classIndex'] = np.where(dfNames['names'].str.contains('facturen'), 1, dfNames['classIndex'])
dfNames['classIndex'] = np.where(dfNames['names'].str.contains('aanmaningen'), 2, dfNames['classIndex'])

Class list:
- 0 = Other
- 1 = invoice
- 2 = Payement reminder

### Display top 20 rows

In [15]:
dfNames.head(20)

,names,contents,classIndex
0,00057d8d-2e28-45d2-8836-e80003cadafa-andere,hier voor de online bijlage van uw van publiek recht nummer is online beschikbaar Beste informeren u dat de bijlage van uw van publiek recht met nummer en dd vanaf nu online beschikbaar bijlage downloaden vriendelijke geval van met betrekking tot uw gelieve ons te contacteren via,0
1,00403521-b493-413d-b0a9-db90dd069dad-facturen,uw factuur als bijlage te er ons op of,1
2,005385fa-959e-4a8b-8763-15261c217b41-facturen,bijlage kan u de factuur voor de maand juli vriendelijke groeten Nadine bus blok,1
3,00552116-d17a-452a-b1f3-e69f76a7cef1-facturen,de bijlage ontvangt u onze vragen kunt u terecht bij n van onze medewerkers via telefoonnummer of email vriendelijke M the environment email,1
4,00680439-07fe-4e6d-a145-e339a3d73e40-facturen,bericht bevat uw elektronische elektronisch ondertekend document is de wettelijke het kader van de wetgeving op elektronisch factureren bent u verplicht uw facturen in hun originele elektronische formaat te bewaren gedurende de wettelijk bepaalde Een geprinte versie van de elektronische factuur geldt in geen geval als wettelijke de elektronisch ondertekende factuur te kunnen bekijken en dient u Adobe of hoger te do to is sent to on services contact details,1
5,0070566b-63e7-4cca-82ff-116f1f834130-andere,voor uw is een definitieve volgende stappen voor de ophaling van het uw contract nauwkeurig en onderteken het digitaal via deze link ID hieronder op uw naam om uw identiteit te er meerdere Dan verifieert de hoofdverantwoordelijke Sam uw naam in het Dan hoeft u niets meer te betaalt op factuur met een termijn van u zelf niet deze mail door naar de ophaler als kan uw materiaal oppikken in Gent volgens de uren aangeduid in het stappen moeten voltooid zijn factuur ontvangt u per mail na inlevering en controle van het is een definitieve kan enkel kosteloos binnen de na verzending van deze zijn er kosten aan zie onze algemene voorwaarden voor meer To email are to click,0
6,0076fa2e-db45-4b1f-bd08-dc73ae26b42b-andere,jullie Daan Ceulemans toe als leverancier de firmanaam is Kil Niel Hilde Sijbers planner DPG Vilvoorde M,0
7,007e144d-da55-43e1-938c-d94ee76d43c7-aanmaningen,bijlage onze voor factuur van jullie die hierop betrekking heeft is F nv Kasteelstraat Tielt hier onze catalogus Facebook Instagram Twitter LinkedIn email an the delete all monitor email to Jerry vrijdag augustus facturen DPG Beste zou het kunnen dat we de factuur van waarvan mijn collega Kristof al sprak nog niet hebben mogen mag naar verzonden worden met mij in kopie van de mail vriendelijke Jerry accountant DPG Antwerpen Jun at terugbetaling is vrijdag dus zal hoogstwaarschijnlijk vandaag bij jullie de van excl zal ik achter mijn collega haar veren zitten dag F nv Kasteelstraat Tielt hier onze catalogus Facebook Instagram Twitter LinkedIn email an the delete all monitor email to Kristof donderdag juni Pieter Clio Janssens Sandra Stefanie Jerry facturen DPG Dag Pieter en worden opgemaakt bij de volgende van de op factuur was er inderdaad iets het zou inderdaad het makkelijkste zijn als de terugbetaling gebeurt van deze zie ik nog dat we jullie van ontbreken Kunnen jullie deze nog Kristof accountant DPG Antwerpen Jun at Clio Janssens Dag voor je snelle Kristof jij bijgevoegde mail aan even kunnen bekijken vriendelijke Clio Janssens DPG Antwerpen Jun at facturen DPG Stefanie Sandra je zo snel mogelijk werk maken van de Dag heb even gepolst bij mijn collega wat er contractueel bepaald is van vervaltermijn voor factuur laat hier zo snel mogelijk iets over er ook nog even kunnen gekeken worden voor de die wij nog van jullie moeten mail in Pieter F nv Kasteelstraat Tielt hier onze catalogus Facebook Instagram Twitter LinkedIn email an the delete all monitor email to Clio Janssens woensdag juni Pieter Fabienne facturen DPG hebben nog geen reactie ontvangen op onderstaande mail en wij hebben gemerkt dat uw factuur nog steeds onbetaald Kunnen jullie hiervoor het nodige doen 

### Save dataframe to csv file

In [10]:
dfNames.to_csv('test_extraction_emails.csv')

# Conclusion:
E-mails need a lot of cleaning to extract just the text and leave metacharacters (such as HTML or escape characters) out of the processed results. Beyond that we need more specific preprocessing steps, determined by the model that it will feed through. Unsupervised models will need stopword removal and lemmitazation/stemming to achieve higher performance, while transfer learning models like BERT will be hurt by these preprocessing steps.

#### Sources:
- https://stackoverflow.com/questions/8115261/how-to-remove-all-the-escape-sequences-from-a-list-of-strings
- https://enjoylifescience.com/2020/11/05/analyzing-emails-in-python/
- https://stackoverflow.com/questions/11331982/how-to-remove-any-url-within-a-string-in-python
- https://towardsdatascience.com/remove-personal-information-from-text-with-python-232cb69cf074
- https://monkeylearn.com/blog/text-cleaning/#:~:text=Text%20cleaning%20can%20be%20performed,words%20to%20their%20root%20form.&text=You'd%20need%20to%20perform,Removing%20Stopwords
- https://www.datacamp.com/tutorial/stemming-lemmatization-python
- https://www.projectpro.io/recipes/use-spacy-lemmatizer
- https://pypi.org/project/dutch-words/ (Original dictionary, replaced by Opentaal wordlist)
- https://github.com/OpenTaal/opentaal-wordlist